In [7]:
import numpy as np 
import pandas as pd 

import cv2
from tensorflow import keras
from imgaug import augmenters as ia
import os
import matplotlib.pyplot as plt
import gc; gc.enable()

# here = os.path.dirname(os.path.abspath(__file__))
PATH_IN =  '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/'
PATH_IMG = PATH_IN + "dataset/train_images/"
PATH_OUT = PATH_IN + 'data/train/'
# body

# read in the list of training images
df = pd.read_csv(PATH_IN + "train.csv")
# df = df.head().copy()
gc.collect()

# idiot-test with first image
id_code = df['id_code'].values[0]
label = df['diagnosis'].values[0]
labels = {}
#
# SKIP
# do the thing... (loop)
flip_h = ia.Fliplr(1)
flip_v = ia.Flipud(1)
g_blur = ia.GaussianBlur(sigma=(0.0, 5.0))
rotate = ia.Affine(rotate=(-180, 180))
shear = ia.Affine(shear=(-5, 5))


In [2]:
def get_img(id_code):
    img_path = PATH_IMG+id_code+'.png'
    img = cv2.imread(img_path)
#     plt.imshow(img)
    return img

def getRadius(img):
    circles = cv2.HoughCircles(img,3,1,max(img.shape)/2,param1=50,param2=30,minRadius= int(max(img.shape)/6),maxRadius=max(img.shape))
    return circles[0][0][2]

In [8]:
for ind in df.index:
    id_code = df['id_code'][ind]
    diag = df['diagnosis'][ind]
    img_array = get_img(id_code)
    #crop each image to 80% of fundus diameter
    try:
        r = getRadius(img_array)
        crop_r = round(r*0.8) 
    except:
        crop_r = (img_array.shape[1]/2)*0.8
            
    center_x = int(img_array.shape[1]/2)
    center_y = int(img_array.shape[0]/2)
    
    left_border = int(center_x - crop_r)
    right_border = int(center_x + crop_r)
    upper_border = int(center_y - crop_r)
    lower_border = int(center_y + crop_r)
    
    if left_border < 0:
        difference = 0- left_border
        left_border = 0
        right_border -=difference
        upper_border += difference
        lower_border -= difference
    if upper_border < 0:
        difference = 0- upper_border
        upper_border = 0
        lower_border -= difference
        left_border += difference
        right_border -= difference

    cropped = img_array[upper_border:lower_border, left_border:right_border]
    #resize image to resnet's expected input size
    img = cv2.resize(cropped, (224,224), interpolation = cv2.INTER_CUBIC)
    
    h = id_code +'_h'
    v = id_code + '_v'
    rot = id_code +'_rot'
    sh = id_code + '_sh'
    blur = id_code + '_blur'   
    subfolder = '/'+str(diag)+'/'
    cv2.imwrite(PATH_OUT+subfolder + h + ".png",flip_h.augment_image(img))
    cv2.imwrite(PATH_OUT+subfolder + v + ".png",flip_v.augment_image(img))
    cv2.imwrite(PATH_OUT+subfolder + rot+ '.png',rotate.augment_image(img))
    cv2.imwrite(PATH_OUT+subfolder + sh + ".png",shear.augment_image(img))
    cv2.imwrite(PATH_OUT+subfolder + blur + '.png',g_blur.augment_image(img)) 
    cv2.imwrite(PATH_OUT+subfolder + id_code + '.png', img)
    labels.update(dict.fromkeys([h,v,rot,sh,blur, id_code],diag))
    # for each augmentation
        # save new image
        # update new list of labels

In [9]:
lbls = pd.DataFrame()
lbls = lbls.from_dict(labels, orient='index', columns = ['diagnosis'])

In [10]:
lbls.to_csv(PATH_IN+'drlabels.csv')